##### Jupyter Notebook, Step 4 - Build Model
- Implement your final model
- (Optionally) use the entire data set

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from tqdm import tqdm
import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
# from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import (precision_score, 
                             accuracy_score, 
                             roc_auc_score, 
                             roc_curve, 
                             precision_recall_curve, 
                             recall_score
#                              make_scorer,
#                              auc,
#                              classification_report,
#                              confusion_matrix
                            )
# !conda install psycopg2 --yes
# import psycopg2 as pg2
# from psycopg2.extras import RealDictCursor
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [ ]:
# draw all the data for just our 20 features

# run the models above a certain threshold on these, w/ a held-out test set

In [ ]:
# import UCI madelon data and set the nan column to labels

def import_and_labels(data, labels):
    
    data = data
    labels = labels
    madelon_train = pd.read_csv(data, delimiter=' ', header=None)
    madelon_labels = pd.read_csv(labels, delimiter=' ', header=None)
    madelon_train[500] = madelon_labels

    return madelon_train

madelon_train = import_and_labels('madelon_train.data.csv','madelon_train.labels.csv')

In [ ]:
UCIsample_list = [28,48,64,105,128,153,241,281,318,336,338,378,433,442,451,453,455,472,475,493,500]
DBsample_list = ['feat_257','feat_269','feat_308','feat_315',\
                 'feat_336','feat_341','feat_395','feat_504','feat_526','feat_639','feat_681',\
                 'feat_701','feat_724','feat_736','feat_769','feat_808','feat_829','feat_867','feat_920',\
                 'feat_956','target']

def drop_noise(df, signal_list):
    temp_df = df
    for column in temp_df.columns:
        if column not in signal_list:
            temp_df.drop(column, axis=1, inplace=True)
    return temp_df

In [ ]:
UCIfull_clean = drop_noise(madelon_train, UCIsample_list)
UCIfull_clean.head()

In [ ]:
pickle.dump(UCIfull_clean, open('UCIfull_clean', "wb" ) )


In [ ]:
def con_cur_to_class_db():
    con = pg2.connect(host='34.211.227.227',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

def draw_sample():
    con, cur = con_cur_to_class_db()
    cur.execute('SELECT feat_257,feat_269,feat_308,feat_315,feat_336,feat_341,feat_395,feat_504,feat_526,feat_639,feat_681,feat_701,feat_724,feat_736,feat_769,feat_808,feat_829,feat_867,feat_920,feat_956,target FROM madelon;')
    mad_db = cur.fetchall()
    con.close()
    return pd.DataFrame(mad_db)

In [ ]:
DBfull_clean = draw_sample()
pickle.dump(DBfull_clean, open('DBfull_clean', "wb" ) )

In [ ]:
DBfull_clean.shape

In [ ]:
DBfull_clean = pickle.load( open( "DBfull_clean", "rb" ) )

# drop the seemingly unneeded ID column
# DBfull_clean.drop('_id', axis=1, inplace=True)
DBfull_clean.head()

In [8]:
# define my controlling function and pipelines to gridsearch

DBfull_clean = pickle.load( open( "DBfull_clean", "rb" ) )
UCIfull_clean = pickle.load( open( "UCIfull_clean", "rb" ) )

def score_pipelines(sample_list, model_zip):
    results = []
    for sample in sample_list:
        y = sample.iloc[:,-1]
        X = sample.iloc[:,0:-1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.25, stratify=y)
        for model_name, model in tqdm(model_zip):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            recall = recall_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            results.append({
                    'sample': sample.name,
                    'name':'{}'.format(model_name),
                    'model': model,
                    'best_params': model.best_params_,
                    'train_accuracy': model.score(X_train, y_train),
                    'test_accuracy': model.score(X_test, y_test),
                    'recall': recall,
                    'precision': precision,
            })
    return pd.DataFrame(results)

knn = Pipeline([
    ('scaling', StandardScaler()), 
#     ('scaling', StandardScaler(with_mean=False)), 
    ('pca', PCA(n_components=5)),
    ('classifier', KNeighborsClassifier())])

svc = Pipeline([
    ('scaling', StandardScaler()), 
#     ('scaling', StandardScaler(with_mean=False)),     
    ('pca', PCA(n_components=5)),
    ('classifier', SVC())])

bag = Pipeline([
    ('scaling', StandardScaler()), 
#     ('scaling', StandardScaler(with_mean=False)),     
    ('pca', PCA(n_components=5)),
    ('clf', BaggingClassifier(DecisionTreeClassifier(random_state=42), max_samples=.8, random_state=42))])

rfc = Pipeline([
    ('scaling', StandardScaler()), 
#     ('scaling', StandardScaler(with_mean=False)),     
    ('pca', PCA(n_components=5)),
    ('clf',RandomForestClassifier(random_state=42))])

etc = Pipeline([
    ('scaling', StandardScaler()), 
#     ('scaling', StandardScaler(with_mean=False)),     
    ('pca', PCA(n_components=5)),
    ('clf',ExtraTreesClassifier(random_state=42))])

knn_params = {
              'classifier__weights':['distance'],
              'classifier__n_neighbors':np.arange(3,13,2)}

svc_params = {
               'classifier__gamma':np.logspace(-3,3,7), 
               'classifier__C':np.logspace(-3,3,7),
               'classifier__kernel':['rbf']}

bag_params = {
    'clf__base_estimator': [DecisionTreeClassifier(max_depth=md, random_state=42) for md in [5,7,10,None]],
    'clf__n_estimators':[10,50,100]}

rfc_params = {
    'clf__n_estimators':[10,50,100,200],
    'clf__max_features':['auto','log2']}

etc_params = {
    'clf__bootstrap':[True, False],
    'clf__n_estimators':[10,50,100,200]}

bag_gs = GridSearchCV(bag, bag_params, cv=5, n_jobs=-1, verbose=1)
rfc_gs = GridSearchCV(rfc, rfc_params, cv=5, n_jobs=-1, verbose=1)
etc_gs = GridSearchCV(etc, etc_params, cv=5, n_jobs=-1, verbose=1)
knn_gs = GridSearchCV(knn, knn_params, cv=5, n_jobs=-1, verbose=1)
svc_gs = GridSearchCV(svc, svc_params, cv=5, n_jobs=-1, verbose=1)

UCIfull_clean.name = 'UCIfull_clean'
DBfull_clean.name = 'DBfull_clean' 

# drop pipes that did poorly in gridsearching after tuning: logr, dct, 
pipe_names = ['knn_gs', 'svc_gs', 'bag_gs', 'rfc_gs', 'etc_gs']
pipe_list = [knn_gs, svc_gs, bag_gs, rfc_gs, etc_gs]
model_zip = list(zip(pipe_names, pipe_list))


In [9]:
sample_list = [UCIfull_clean]
results = score_pipelines(sample_list, model_zip)
display(results.sort_values('test_accuracy', ascending=False))

  0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/base.py:80: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/base.py:80: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/base.py:80: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/base.py:80: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)



JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run

In [ ]:
for i in range(0,5):
    display(results.best_params.iloc[i])
    
# using some of these lessons, modify for the full DB run

In [ ]:
# adjusting the parameters based on which ones crash AWS and which don't...

knn_params = {
              'classifier__weights':['distance'],
              'classifier__n_neighbors':np.arange(5,11,2)}

svc_params = {
               'classifier__gamma':np.logspace(-2,2,3), 
               'classifier__C':np.logspace(-2,2,3),
               'classifier__kernel':['rbf']}

rfc_params = {
                'clf__n_estimators':[100],
                'clf__max_features':['auto']}

svc_gs = GridSearchCV(svc, svc_params, cv=3, n_jobs=-1, verbose=10)
knn_gs = GridSearchCV(knn, knn_params, cv=3, n_jobs=-1, verbose=10)
rfc_gs = GridSearchCV(rfc, rfc_params, cv=3, n_jobs=-1, verbose=10)

pipe_names = ['svc_gs', 'knn_gs', 'rfc_gs']
pipe_list = [svc_gs, knn_gs, rfc_gs]
model_zip = list(zip(pipe_names, pipe_list))

sample_list = [DBfull_clean]
results = score_pipelines(sample_list, model_zip)
display(results.sort_values('test_accuracy', ascending=False))

In [ ]:
for i in range(0,1):
    display(results.best_params.iloc[i])

In [ ]:
# etc_params = {
# #     'clf__bootstrap':[True, False],
#     'clf__n_estimators':[100,200]}

rfc_params = {
    'clf__n_estimators':[100,200],
    'clf__max_features':['auto']}

# etc_gs = GridSearchCV(etc, etc_params, cv=3, n_jobs=-1, verbose=10)
rfc_gs = GridSearchCV(rfc, rfc_params, cv=3, n_jobs=-1, verbose=10)

pipe_names = ['rfc_gs']
pipe_list = [rfc_gs]
model_zip = list(zip(pipe_names, pipe_list))

sample_list = [DBfull_clean]
results = score_pipelines(sample_list, model_zip)
display(results.sort_values('test_accuracy', ascending=False))